In [ ]:
import sys
base_path='' 
sys.path.append(base_path)
sys.path.append('/home/bsc')
import mixturemm

workdir = 'glyc_blieck_egorov'

In [ ]:
project = mixturemm.Project(workdir,
                            simulation_platform='CUDA',
                            simulation_properties={'DeviceIndex': '0', 'Precision': 'double'},
                            total_number_molecules=[700, 3000, 6000],
                            init_box_side_length=[40, 70, 90],
                            chi_water_s=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                            temperature_s=[288.15, 293.15, 298.15, 303.15],
                            npt_equilibration_pressure_s=[1],
                            npt_equilibration_pressure_coupling_frequency=500,
                            npt_equilibration_temperature_coupling_frequency=0.1,
                            npt_equilibration_timestep_fs=2,
                            npt_equilibration_duration_ns=12,
                            reporting_frequency_state_npt_equilibration = 500,
                            nvt_equilibration_temperature_coupling_frequency=0.1,
                            nvt_equilibration_timestep_fs=2,
                            nvt_equilibration_duration_ns=20,
                            reporting_frequency_state_nvt_equilibration=4000,
                            nve_production_timestep_fs=1,
                            nve_production_duration_ns=50,
                            reporting_frequency_coordinates_unwrapped=4000,
                            reporting_frequency_coordinates_wrapped=8000,
                            reporting_frequency_state_nve_production=4000,
                            replica_count=20,
                            pme_error_tolerance=0.000001,
                            constraint_tolerance=0.0000001,
                            cutoff_distance_nm=1.1,
                            cutoff_switch_distance_nm=1.0)

In [ ]:
Water = mixturemm.Molecule(workdir, name='Water', number_of_atoms=4, molar_ratio=1, path_to_pdb=f'{base_path}input_files/molecules/Water.pdb', abbreviation='HOH', smiles='O', inchi='1S/H2O/h1H2', molar_mass=18.015, use_as_water=True)
Glycerol = mixturemm.Molecule(workdir, 'Glycerol', 14, 1, f'{base_path}input_files/molecules/Glycerol.pdb', 'Gl', 'C(C(CO)O)O', '1S/C3H8O3/c4-1-3(6)2-5/h3-6H,1-2H2', 92.09382)

In [ ]:

waterFF = mixturemm.Forcefield(workdir, f'{base_path}input_files/forcefield/tip4p-fb.xml','tip4p-fb', built_in=False)
glycFF = mixturemm.Forcefield(workdir, f'{base_path}input_files/forcefield/glyc_blieck_egorov.xml','blieck_egorov', built_in=False)
project.add_molecule(Water)
project.add_molecule(Glycerol)
project.add_forcefield(waterFF)
project.add_forcefield(glycFF)

In [ ]:



project.create_simulationboxes()
project.create_systems()
project.create_replicas(start=1)



In [ ]:
project.create_hpc_submission_adjust_to_correct_density(
hpc_workspace="/hkfs/work/workspace/scratch/st_st167271-des",
hpc_folder="/hkfs/work/workspace/scratch/st_st167271-des/glyc_blieck_egorov",
hpc_scripts_folder="/hkfs/work/workspace/scratch/st_st167271-des/",
environment_name="des",
scheduler='SBATCH',
partition = 'accelerated',
number_of_threads=152,
number_of_gpus=4,
chunk_size=4,
max_runtime_hh_mm_ss='24:00:00',
which_hpc="HoreKa",
chain_submission_number=1,
dependency_type='afternotok',
checkpoint_frequency=10000)


In [ ]:
project.create_hpc_submission_simulate(environment_name='des',
                            scheduler='SBATCH',
                            max_runtime_hh_mm_ss='24:00:00',
                            number_of_threads=152,
                            partition='accelerated',
                            chain_submission_number=1,
                            number_of_gpus=4,
                            chunk_size=4,                              
                            dependency_type='afternotok',
                            checkpoint_frequency=500000,
                            hpc_workspace="/hkfs/work/workspace/scratch/st_st167271-des",
                            hpc_folder="/hkfs/work/workspace/scratch/st_st167271-des/glyc_blieck_egorov",
                            hpc_scripts_folder="/hkfs/work/workspace/scratch/st_st167271-des",
                            which_hpc='HOREKA',
                            max_number_of_jobs=50)


In [ ]:
project.hpc_extend_replica_folders("/hkfs/work/workspace/scratch/st_st167271-des/glyc_blieck_egorov")


In [ ]:
project.remove_done_jobs_from_replicas(f'{workdir}/job_checks.txt')

In [ ]:
project.hpc_job_checker(hpc_folder='/hkfs/work/workspace/scratch/st_st167271-des/glyc_blieck_egorov', 
    hpc_scripts_folder='/hkfs/work/workspace/scratch/st_st167271-des', 
    hpc_workspace='/hkfs/work/workspace/scratch/st_st167271-des', 
    environment_name='des', 
    output='verbose',
    energy_shift_tolerance_percent=1)

In [ ]:
project.create_hpc_submission_analyze_density(hpc_workspace='/hkfs/work/workspace/scratch/st_st167271-des',
    hpc_folder='/hkfs/work/workspace/scratch/st_st167271-des/glyc_blieck_egorov',
    hpc_scripts_folder='/hkfs/work/workspace/scratch/st_st167271-des',
    environment_name='analysis',
    scheduler='SBATCH',
    partition='cpuonly',
    number_of_threads=152,
    max_runtime_hh_mm_ss='00:10:00',
    which_hpc='HoreKa'
    )

In [ ]:
project.create_hpc_submission_analyze_msd(hpc_workspace='/hkfs/work/workspace/scratch/st_st167271-des',
    hpc_folder='/hkfs/work/workspace/scratch/st_st167271-des/glyc_blieck_egorov',
    hpc_scripts_folder='/hkfs/work/workspace/scratch/st_st167271-des',
    environment_name='analysis',
    scheduler='SBATCH',
    partition='cpuonly',
    number_of_threads=152,
    parallel_running=4,
    submission_split=50,
    max_runtime_hh_mm_ss='24:00:00',
    which_hpc='HoreKa',
    fit_starting_percentage=20,
    fit_ending_percentage=80,
    just_conclude=False
    )